#mount driver

In [1]:
from google.colab import drive

drive.mount('/content/drive')

!ls "/content/drive/My Drive/Colab Notebooks/Data"

!cp -r '/content/drive/My Drive/Colab Notebooks/Data' "./data"

!ls

Mounted at /content/drive
result2.csv  result3.csv  result.csv  result.gsheet  test.csv  train.csv
cp: cannot open '/content/drive/My Drive/Colab Notebooks/Data/result.gsheet' for reading: Operation not supported
data  drive  sample_data


#import libraries

In [106]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

#Import csv data

In [143]:
# Import train dataset as a dataframe
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/train.csv",parse_dates = ['date'],dayfirst=True)

In [35]:
#Create a new field named 'date_only' which contains only the date without time
df['date_only'] = df['date'].dt.normalize()

In [36]:
df.head(5)

,id,date,speed,date_year,date_month,date_day,date_hour,date_only
0,0,2017-01-01 00:00:00,43.002930,2017,1,1,0,2017-01-01
1,1,2017-01-01 01:00:00,46.118696,2017,1,1,1,2017-01-01
2,2,2017-01-01 02:00:00,44.294158,2017,1,1,2,2017-01-01
3,3,2017-01-01 03:00:00,41.067468,2017,1,1,3,2017-01-01
4,4,2017-01-01 04:00:00,46.448653,2017,1,1,4,2017-01-01


In [115]:
#Import the test dataset for the prediction
z = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/test.csv",parse_dates = ['date'],dayfirst=True)
#Create a new field named 'date_only' which contains only the date without time
z['date_only'] = z['date'].dt.normalize()

#Separate into Year, Month, Day and Hour

In [21]:
#Create function to split into 4 new columns (year, month, day, hour)
def extract_date(df, column):
    df[column+"_year"] = df[column].apply(lambda x: x.year)
    df[column+"_month"] = df[column].apply(lambda x: x.month)
    df[column+"_day"] = df[column].apply(lambda x: x.day)
    df[column+"_hour"] = df[column].apply(lambda x: x.hour)

In [22]:
extract_date(df, 'date')

In [56]:
df.head(5)

,id,date,speed,date_year,date_month,date_day,date_hour,date_only
0,0,2017-01-01 00:00:00,43.002930,2017,1,1,0,2017-01-01
1,1,2017-01-01 01:00:00,46.118696,2017,1,1,1,2017-01-01
2,2,2017-01-01 02:00:00,44.294158,2017,1,1,2,2017-01-01
3,3,2017-01-01 03:00:00,41.067468,2017,1,1,3,2017-01-01
4,4,2017-01-01 04:00:00,46.448653,2017,1,1,4,2017-01-01


In [118]:
extract_date(z, 'date')
z.head(5)

,id,date,date_only,date_year,date_month,date_day,date_hour
0,0,2018-01-01 02:00:00,2018-01-01,2018,1,1,2
1,1,2018-01-01 05:00:00,2018-01-01,2018,1,1,5
2,2,2018-01-01 07:00:00,2018-01-01,2018,1,1,7
3,3,2018-01-01 08:00:00,2018-01-01,2018,1,1,8
4,4,2018-01-01 10:00:00,2018-01-01,2018,1,1,10


#Get Public Holiday


In [27]:
#Create a function to get the Hong Kong public holiday from the government website
import requests
from bs4 import BeautifulSoup

def get_hk_public_holiday(in_year):

    url = 'https://www.gov.hk/en/about/abouthk/holiday/'+str(in_year)+'.htm'
    page = requests.get(url)
    # print(response.text)

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(page.text, 'html.parser')
    # print(soup.prettify())

    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]

    df[1] = df[1] + " " +str(in_year)

    return pd.to_datetime(df[1], dayfirst=True)[1:]

In [28]:
get_hk_public_holiday(2017)

1    2017-01-02
2    2017-01-28
3    2017-01-30
4    2017-01-31
5    2017-04-04
6    2017-04-14
7    2017-04-15
8    2017-04-17
9    2017-05-01
10   2017-05-03
11   2017-05-30
12   2017-07-01
13   2017-10-02
14   2017-10-05
15   2017-10-28
16   2017-12-25
17   2017-12-26
Name: 1, dtype: datetime64[ns]

In [29]:
s2017 = pd.Series(get_hk_public_holiday(2017),
              name="date")
s2018 = pd.Series(get_hk_public_holiday(2018),
              name="date")

,date
1,2018-01-01
2,2018-02-16
3,2018-02-17
4,2018-02-19
5,2018-03-30
6,2018-03-31
7,2018-04-02
8,2018-04-05
9,2018-05-01
10,2018-05-22


In [30]:
holiday = pd.concat([s2017,s2018])

In [31]:
#Create a dataframe to combine both 2017 and 2018 public holidays
holiday = holiday.to_frame()

In [32]:
#Assign the value '1' as a flag to indicate the public holiday
holiday['value'] = 1

In [33]:
print(holiday)

         date  value
1  2017-01-02      1
2  2017-01-28      1
3  2017-01-30      1
4  2017-01-31      1
5  2017-04-04      1
6  2017-04-14      1
7  2017-04-15      1
8  2017-04-17      1
9  2017-05-01      1
10 2017-05-03      1
11 2017-05-30      1
12 2017-07-01      1
13 2017-10-02      1
14 2017-10-05      1
15 2017-10-28      1
16 2017-12-25      1
17 2017-12-26      1
1  2018-01-01      1
2  2018-02-16      1
3  2018-02-17      1
4  2018-02-19      1
5  2018-03-30      1
6  2018-03-31      1
7  2018-04-02      1
8  2018-04-05      1
9  2018-05-01      1
10 2018-05-22      1
11 2018-06-18      1
12 2018-07-02      1
13 2018-09-25      1
14 2018-10-01      1
15 2018-10-17      1
16 2018-12-25      1
17 2018-12-26      1


#Merge Holiday

In [40]:
#Merge the holiday dataframe to the training dataset
df2= pd.merge(df,holiday, right_on='date',left_on='date_only',how='left')
df2 = df2.drop('date_y', 1)
df2 = df2.rename(columns={'date_x': 'date'})
df2['value'] = df2['value'].fillna(0)
df2 = df2.rename(columns={'value': 'Flag_Holiday'})
print(df2)

          id              date_x      speed  ...  date_hour  date_only  value
0          0 2017-01-01 00:00:00  43.002930  ...          0 2017-01-01    0.0
1          1 2017-01-01 01:00:00  46.118696  ...          1 2017-01-01    0.0
2          2 2017-01-01 02:00:00  44.294158  ...          2 2017-01-01    0.0
3          3 2017-01-01 03:00:00  41.067468  ...          3 2017-01-01    0.0
4          4 2017-01-01 04:00:00  46.448653  ...          4 2017-01-01    0.0
...      ...                 ...        ...  ...        ...        ...    ...
14001  14001 2018-12-31 12:00:00  19.865269  ...         12 2018-12-31    0.0
14002  14002 2018-12-31 15:00:00  17.820375  ...         15 2018-12-31    0.0
14003  14003 2018-12-31 16:00:00  12.501851  ...         16 2018-12-31    0.0
14004  14004 2018-12-31 18:00:00  15.979319  ...         18 2018-12-31    0.0
14005  14005 2018-12-31 20:00:00  40.594183  ...         20 2018-12-31    0.0

[14006 rows x 9 columns]


In [49]:
#Change the field type to integer
df2['Flag_Holiday'] = pd.to_numeric(df2['Flag_Holiday'], downcast='integer')

In [50]:
df2.dtypes

id                       int64
date            datetime64[ns]
speed                  float64
date_year                int64
date_month               int64
date_day                 int64
date_hour                int64
date_only       datetime64[ns]
Flag_Holiday              int8
dtype: object

In [119]:
#Merge the holiday dataframe to the testing dataset
z2= pd.merge(z,holiday, right_on='date',left_on='date_only',how='left')
z2 = z2.drop('date_y', 1)
z2 = z2.rename(columns={'date_x': 'date'})
z2['value'] = z2['value'].fillna(0)
z2 = z2.rename(columns={'value': 'Flag_Holiday'})
#Change the field type to integer
z2['Flag_Holiday'] = pd.to_numeric(z2['Flag_Holiday'], downcast='integer')
print(z2)

        id                date  date_only  ...  date_day  date_hour  Flag_Holiday
0        0 2018-01-01 02:00:00 2018-01-01  ...         1          2             1
1        1 2018-01-01 05:00:00 2018-01-01  ...         1          5             1
2        2 2018-01-01 07:00:00 2018-01-01  ...         1          7             1
3        3 2018-01-01 08:00:00 2018-01-01  ...         1          8             1
4        4 2018-01-01 10:00:00 2018-01-01  ...         1         10             1
...    ...                 ...        ...  ...       ...        ...           ...
3499  3499 2018-12-31 17:00:00 2018-12-31  ...        31         17             0
3500  3500 2018-12-31 19:00:00 2018-12-31  ...        31         19             0
3501  3501 2018-12-31 21:00:00 2018-12-31  ...        31         21             0
3502  3502 2018-12-31 22:00:00 2018-12-31  ...        31         22             0
3503  3503 2018-12-31 23:00:00 2018-12-31  ...        31         23             0

[3504 rows x 8 

In [120]:
z2.dtypes

id                       int64
date            datetime64[ns]
date_only       datetime64[ns]
date_year                int64
date_month               int64
date_day                 int64
date_hour                int64
Flag_Holiday              int8
dtype: object

#Get weekday names

In [71]:
#Get the weekday name for the training dataset
df2['Week'] = df2['date_only'].dt.day_name()

In [76]:
df2.head(5)

,id,date,speed,date_year,date_month,date_day,date_hour,date_only,Flag_Holiday,Week
0,0,2017-01-01 00:00:00,43.002930,2017,1,1,0,2017-01-01,0,Sunday
1,1,2017-01-01 01:00:00,46.118696,2017,1,1,1,2017-01-01,0,Sunday
2,2,2017-01-01 02:00:00,44.294158,2017,1,1,2,2017-01-01,0,Sunday
3,3,2017-01-01 03:00:00,41.067468,2017,1,1,3,2017-01-01,0,Sunday
4,4,2017-01-01 04:00:00,46.448653,2017,1,1,4,2017-01-01,0,Sunday


In [80]:
#One-hot encoding
df2 = pd.get_dummies(df2, prefix='Week')

In [85]:
df2.head(5)

,id,date,speed,date_year,date_month,date_day,date_hour,date_only,Flag_Holiday,Week_Friday,Week_Monday,Week_Saturday,Week_Sunday,Week_Thursday,Week_Tuesday,Week_Wednesday
0,0,2017-01-01 00:00:00,43.002930,2017,1,1,0,2017-01-01,0,0,0,0,1,0,0,0
1,1,2017-01-01 01:00:00,46.118696,2017,1,1,1,2017-01-01,0,0,0,0,1,0,0,0
2,2,2017-01-01 02:00:00,44.294158,2017,1,1,2,2017-01-01,0,0,0,0,1,0,0,0
3,3,2017-01-01 03:00:00,41.067468,2017,1,1,3,2017-01-01,0,0,0,0,1,0,0,0
4,4,2017-01-01 04:00:00,46.448653,2017,1,1,4,2017-01-01,0,0,0,0,1,0,0,0


In [121]:
#Get the weekday name for the testing dataset
z2['Week'] = z2['date_only'].dt.day_name()
#One-hot encoding
z2 = pd.get_dummies(z2, prefix='Week')

In [122]:
z2.head(5)

,id,date,date_only,date_year,date_month,date_day,date_hour,Flag_Holiday,Week_Friday,Week_Monday,Week_Saturday,Week_Sunday,Week_Thursday,Week_Tuesday,Week_Wednesday
0,0,2018-01-01 02:00:00,2018-01-01,2018,1,1,2,1,0,1,0,0,0,0,0
1,1,2018-01-01 05:00:00,2018-01-01,2018,1,1,5,1,0,1,0,0,0,0,0
2,2,2018-01-01 07:00:00,2018-01-01,2018,1,1,7,1,0,1,0,0,0,0,0
3,3,2018-01-01 08:00:00,2018-01-01,2018,1,1,8,1,0,1,0,0,0,0,0
4,4,2018-01-01 10:00:00,2018-01-01,2018,1,1,10,1,0,1,0,0,0,0,0


#Split Test and Validation set

In [88]:
#Separate into features and labels
non_feature_col = ['id','speed','date','date_only']
X = df2[[col for col in df2.columns if col not in non_feature_col]]
y = df2['speed']

In [89]:
X.head(1)

,date_year,date_month,date_day,date_hour,Flag_Holiday,Week_Friday,Week_Monday,Week_Saturday,Week_Sunday,Week_Thursday,Week_Tuesday,Week_Wednesday
0,2017,1,1,0,0,0,0,0,1,0,0,0


In [90]:
y.head(1)

0    43.00293
Name: speed, dtype: float64

In [93]:
#Split into train and validation datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Random Forest Regressor (w/o tuning parameter)

In [94]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Validating Features Shape:', X_test.shape)
print('Validating Labels Shape:', y_test.shape)

Training Features Shape: (11204, 12)
Training Labels Shape: (11204,)
Testing Features Shape: (2802, 12)
Testing Labels Shape: (2802,)


In [98]:
# Instantiate model
rf = RandomForestRegressor(n_estimators=100, max_depth=20, min_samples_leaf=3, random_state=42, n_jobs=-1)
# Train the model on training data
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=3,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [104]:
#test error
MSE_score = mean_squared_error(y_test, rf.predict(X_test)) 

In [105]:
MSE_score

11.350065276376442

#Grid Search CV --> tune hyper-parameters

In [107]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [100, 200], 
                     'max_depth': [15,20],
                     'min_samples_leaf':[3, 5],
                     'n_jobs':[-1]
                     }]

In [108]:
rf = RandomForestRegressor()

# Use ShuffleSplit to do the cross validation
cv_split = ShuffleSplit(n_splits=5, train_size=0.8, test_size=0.2)

#Use GridSearchCV to tune hyper parameters
clf_rf = GridSearchCV(rf, tuned_parameters, n_jobs=-1, cv=cv_split, scoring='neg_mean_squared_error')

clf_rf.fit(X_train, y_train)

print(clf_rf.best_params_)
print(-1*clf_rf.best_score_ )

{'max_depth': 20, 'min_samples_leaf': 3, 'n_estimators': 200, 'n_jobs': -1}
13.440259069736621


In [109]:
#test error
MSE_score = mean_squared_error(y_test, clf_rf.predict(X_test))  

In [110]:
MSE_score

11.289904820707319

#Random Forest Regressor (after tuning parameter)

In [111]:
# Instantiate model using best hyper parameters
rf2 = RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_leaf=3, random_state=42, n_jobs=-1)
# Train the model on training data using best hyper parameters
rf2.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=3,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [112]:
#test error
MSE_score = mean_squared_error(y_test, rf2.predict(X_test))  

In [113]:
MSE_score

11.317290692299645

#Predict test dataset

In [123]:
#Separate into features and labels
non_feature_col_z = ['id','date','date_only']
z3 = z2[[col for col in z2.columns if col not in non_feature_col_z]]

In [130]:
#Predict the results using trained model
z_result = rf2.predict(z3)
z_result = z_result.reshape(3504, 1)

In [131]:
z_result

array([[48.2492566 ],
       [47.31122267],
       [38.16771035],
       ...,
       [44.49319555],
       [39.78432614],
       [41.86657014]])

In [140]:
#Prepare the dataframe for the dataset to be submitted
z_result_2 = pd.DataFrame(range(0,3504), columns=['id'])
z_result_2['speed'] = pd.DataFrame(z_result,columns=['speed'])

In [141]:
z_result_2

,id,speed
0,0,48.249257
1,1,47.311223
2,2,38.167710
3,3,27.659201
4,4,37.645048
...,...,...
3499,3499,13.720031
3500,3500,22.823957
3501,3501,44.493196
3502,3502,39.784326


#Save output to csv


In [142]:
#Store the dataframe as csv file
z_result_2.to_csv ("/content/drive/My Drive/Colab Notebooks/Data/result_20201205.csv", index = False, header=True)